In [1]:
!pip install tensorflow keras matplotlib

In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt
import os

In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = 256

In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Potato/color",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [5]:
# Preprocess dataset
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
dataset = dataset.map(lambda x, y: (preprocess_input(x), y))

In [6]:
# Split dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

In [7]:
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [8]:
# Create base model (MobileNetV2)
base_model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                         include_top=False,
                         weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [9]:
# Freeze base model
base_model.trainable = False

In [10]:
# Add classification head
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(3, activation='softmax')

In [11]:
# Build the model
inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [12]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [13]:
# Train the model
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=10)

Epoch 1/10
54/54 [==============================] - 390s 6s/step - loss: 0.4525 - accuracy: 0.8391 - val_loss: 0.2269 - val_accuracy: 0.9181
Epoch 2/10
54/54 [==============================] - 147s 3s/step - loss: 0.1594 - accuracy: 0.9606 - val_loss: 0.1566 - val_accuracy: 0.9569
Epoch 3/10
54/54 [==============================] - 144s 3s/step - loss: 0.1154 - accuracy: 0.9682 - val_loss: 0.1353 - val_accuracy: 0.9569
Epoch 4/10
54/54 [==============================] - 126s 2s/step - loss: 0.0905 - accuracy: 0.9745 - val_loss: 0.1209 - val_accuracy: 0.9526
Epoch 5/10
54/54 [==============================] - 145s 3s/step - loss: 0.0751 - accuracy: 0.9797 - val_loss: 0.0871 - val_accuracy: 0.9784
Epoch 6/10
54/54 [==============================] - 129s 2s/step - loss: 0.0603 - accuracy: 0.9844 - val_loss: 0.0809 - val_accuracy: 0.9655
Epoch 7/10
54/54 [==============================] - 144s 3s/step - loss: 0.0532 - accuracy: 0.9896 - val_loss: 0.0828 - val_accuracy: 0.9698
Epoch 8/10
54

In [14]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')
print("Accuracy: ", accuracy * 100, "%")

8/8 [==============================] - 23s 2s/step - loss: 0.0475 - accuracy: 0.9922
Test accuracy: 0.9921875
Accuracy:  99.21875 %


In [15]:
# Save the model
model_version = max([int(i) for i in os.listdir("assets") + [0]]) + 1
model.save(f"assets/{model_version}")

In [16]:
model.save("potato_diagnosis.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
